In [ ]:
################ Simple example of word embeedings  ##########################
# It takes 4 words and gets thes embeddings (w1, w2) as it 2d representation #
##############################################################################
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F
import re

In [ ]:
torch.manual_seed(298630698)

In [ ]:
words =["Troll","is", "great","Gymkata"]

In [ ]:

L = ["Troll is great\n", "Gymkata is great\n", "Sam is fat and heavy\n", "Sam is great\n","Dali is small\n","Gymkata is fat\n",
     "Dali is great\n", "Dali is a bit small\n","Sam and Dali are heavy\n"]
dictionary=set()
# writing to file
file1 = open('myfile.txt', 'w')
file1.writelines(L)
file1.close()
 
# Using readlines()
file1 = open('myfile.txt', 'r')
file1 = open('texto3.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    print("Line {}: {}".format(count, line.strip()))
    # using split() method with a regular expression
    # to extract words from string
    res = re.split(r'\W+', line.strip())
    words = []
    for word in res:
        if word:
            words.append(word)
    for word in words:
        #print(word)
        dictionary.add(word)
print(dictionary)
classes= len(dictionary)


In [ ]:
chars = (dictionary)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
xs=[]
ys=[]

In [ ]:
g=[]
i=0
for line in Lines:
    print("frase ",end="")
    print(i,": ",line)
    
    for w in line.split():
     #   print("2 ",w)
        g.append(w)
    for j,l in zip(g,g[1:]):
        print(j,l)
        ix1 = stoi[j]
        ix2 = stoi[l]
        #print(ix1, ix2)
        xs.append(ix1)
        ys.append(ix2)
    i=i+1
    g=[]

In [ ]:
#xs, ys = [], []
#for line in Lines:
#    for j,l in zip(line,line[1:]):
#        print(j,l)
#        ix1 = stoi[j]
#        ix2 = stoi[l]
#        print(ix1, ix2)
#        xs.append(ix1)
#        ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys) 

In [ ]:
xs=xs-1

In [ ]:
ys=ys-1

In [ ]:
xenc = F.one_hot(xs, num_classes=classes).float()

In [ ]:
yenc = F.one_hot(ys, num_classes=classes).float()
yenc

In [ ]:
x =  xenc #torch.tensor(((1,0,0,0,0),(0,1,0,0,0),(0,0,1,0,0),(0,0,0,1,0),(0,0,0,0,1)),requires_grad=False,dtype=torch.float32)
w1 = torch.randn((classes,2),requires_grad=True,dtype=torch.float32)
w2 = torch.randn((2,classes),requires_grad=True,dtype=torch.float32)
y =  yenc #torch.tensor(((0,1,0,0,0),(0,0,1,0,0),(0,0,0,1,0),(0,1,0,0,0),(0,1,0,0,0)) ,requires_grad=False,dtype=torch.float32)

In [ ]:
#x = torch.tensor(((1,0,0,0),(0,1,0,0),(0,0,1,0),(0,0,0,1)),requires_grad=False,dtype=torch.float32)
#w1 = torch.randn((4,2),requires_grad=True,dtype=torch.float32)
#w2 = torch.randn((2,4),requires_grad=True,dtype=torch.float32)
#y = torch.tensor(((0,1,0,0),(0,0,1,0),(0,0,0,1),(0,1,0,0)) ,requires_grad=False,dtype=torch.float32)

In [ ]:
x.to("cuda:0")
w1.to("cuda:0")
w2.to("cuda:0")
y.to("cuda:0")

In [ ]:
w1

In [ ]:
w1.retain_grad()
w2.retain_grad()


In [ ]:

for i in range (3000):
    z1= x@ w1
    z2= z1 @ w2
    f= torch.softmax(z2,dim=1)
    loss = torch.nn.CrossEntropyLoss()
    w1.grad=None
    w2.grad =None
    r=0.1
    output = loss(f, y)
    output.backward()
    w1.data+=-r*w1.grad
    w2.data+=-r*w2.grad
print(output)

In [ ]:
z1= x@ w1
z2= z1 @ w2
f= torch.softmax(z2,dim=1)
print(f)

In [ ]:
p=list(x[0])
max_value = max(p)
max_index = p.index(max_value)
itos

In [ ]:
for h in range(15):
    s=list(f[h])
    max_value = max(s)
    max_index = s.index(max_value)
    print(itos[max_index+1])
    


In [ ]:
x


In [ ]:
w1

In [ ]:
d = w1.tolist()
x1,y1 =list(zip(*d))
plt.scatter(x1,y1)
for i, txt in enumerate(dictionary):
    plt.annotate(txt, (x1[i], y1[i]))
plt.show()

In [ ]:
d

In [ ]:
fig = plt.figure()
plt.plot(w1.detach().numpy(), color='blue')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
fig

In [ ]:
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F
import re

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
print(words)

In [ ]:

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}
print(itos)

In [ ]:
block_size=3
context=[0]*block_size

In [ ]:
context

In [ ]:
X,Y = [],[]
for w in words:
    #print("word: ",w)
    for ch in w+'.':
        #print("char: ",ch)
        ix= stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context) ,"---->", itos[ix])
        context = context[1:] +[ix]

X=torch.tensor(X)
Y=torch.tensor(Y)

                                                            
    


In [ ]:
X


In [ ]:
X.shape

In [ ]:
g= torch.Generator().manual_seed(2147483647)
C= torch.rand((27,2),generator=g)
W1= torch.rand(6,100,generator=g)
b1 = torch.rand(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2 = torch.randn(27,generator=g)
parameters =[C,W1,b1,W2,b2]

In [ ]:
X.shape

In [ ]:

C.to("cuda:0")
W1.to("cuda:0")
b1.to("cuda:0")
W2.to("cuda:0")
b2.to("cuda:0")


In [ ]:
for p in parameters:
    p.requires_grad= True

In [ ]:
sum(p.nelement() for p in parameters)

In [ ]:
for _ in range(1000):
    emb= C[X]
    h=torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2+ b2
    loss= F.cross_entropy(logits,Y)
    print(loss.item())
    for p in parameters:
            p.grad= None
    loss.backward()
    for p in parameters:
        p.data = p.data -0.1* p.grad

print(loss.item())

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data,C[:,1].data,s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(),C[i,1].item(),itos[i],color='white',ha='center',va='center')
plt.grid('minor')

In [ ]:
for i in range(20):
    out=[]
    context = [0]*block_size
    while True:
        emb=C[torch.tensor([context])]
        h= torch.tanh(emb.view(1,-1) @ W1+b1)
        logits = h@W2 +b2
        probs = F.softmax(logits,dim=1)
        ix= torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
    print(''.join(itos[i] for i in out))

In [ ]:
################### GPT #############

In [ ]:
import torch

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
text = open("input.txt").read()
print(text)

In [ ]:
chars = list(sorted(set(text)))
chars

In [ ]:
for i in enumerate(chars):
    print(i)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda word:    [stoi[ch] for ch in word]
decode = lambda entero:  ''.join([itos[i] for i in entero])

In [ ]:
print(encode("perro"))

In [ ]:
print(decode(encode("perro")))

In [ ]:
data= torch.tensor(encode(text),dtype=torch.long)

In [ ]:
print(data[:100])

In [ ]:
n= int(0.9*len(data))
train_data= data[:n]
val_data= data[n:]

In [ ]:
block_size=8
train_data[:block_size+1]

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context= x[:t+1]
    target = y[t]
    print(f"when the input is {context} the target is {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data= train_data if split=="train" else val_data
    ix= torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)

                
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when the input is {context} the target is {target}")
        

In [1]:
################ Embedding ############
import torch 
from torch.nn import Embedding

In [2]:
n_embeddings,dim = 10,4

In [3]:
emb1 = Embedding(n_embeddings, dim)

In [4]:
emb1

Embedding(10, 4)

In [ ]:
emb1.weight

In [ ]:
imp = torch.LongTensor([[1,3],[5,5]])

In [ ]:
emb1(imp)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

In [ ]:
embeds.weight

In [ ]:
class hola():
    def __init__(self,v):
        self.vocal=v
    
    def forward(self,idx):
        return "aa"
        
    def forward1(self,idx):
        return "holaa"
    
    def generate(self,idx):
        f=self(idx)
        return "3"+f
    

d=hola("a")

In [ ]:
d.generate(2)

In [ ]:
class myLayer(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(10,1)
  def forward(self, input_tensor):
    return self.layer1(input_tensor)
model = myLayer()
input_tensor = torch.rand((2,10))
#//treat as callable, which is same as model.forward(tensor)
model(input_tensor)

In [5]:
############# Conditional probaility########

In [6]:
import re

In [7]:
conjunto =set()
file1 = open('texto3.txt', 'r')
Lines = file1.readlines()
Lines2=[]
words=[]
count = 0


In [8]:
for line in Lines:
    line= "<s> " + line + " </s>"
    line  = re.sub(r'\n', '', line)
    Lines2.append(line)
Lines2

['<s> Roger is young </s>',
 '<s> Dali is a dog </s>',
 '<s> Dakota is old </s>',
 '<s> Roger is funny </s>',
 '<s> Marta is small </s>',
 '<s> Magce is fat </s>',
 '<s> Julia is white </s>',
 '<s> Julia is working </s>',
 '<s> Roger sleeps late </s>',
 '<s> Roger is happy </s>',
 '<s> NEC is a company </s>',
 '<s> Huawei is a company </s>',
 '<s> Nokia is a company </s>',
 '<s> Dakota is a dog </s>',
 '<s> Marta is a woman </s>',
 '<s> Magce is a woman </s>',
 '<s> Marta is black </s>']

In [11]:
for line in Lines2:
    count += 1
    
    for word in line.split():
        words.append(word)
        conjunto.add(word)
        print(word)
print("-> ",conjunto)
classes= len(conjunto)

<s>
Roger
is
young
</s>
<s>
Dali
is
a
dog
</s>
<s>
Dakota
is
old
</s>
<s>
Roger
is
funny
</s>
<s>
Marta
is
small
</s>
<s>
Magce
is
fat
</s>
<s>
Julia
is
white
</s>
<s>
Julia
is
working
</s>
<s>
Roger
sleeps
late
</s>
<s>
Roger
is
happy
</s>
<s>
NEC
is
a
company
</s>
<s>
Huawei
is
a
company
</s>
<s>
Nokia
is
a
company
</s>
<s>
Dakota
is
a
dog
</s>
<s>
Marta
is
a
woman
</s>
<s>
Magce
is
a
woman
</s>
<s>
Marta
is
black
</s>
->  {'working', 'happy', 'Julia', 'Roger', 'dog', 'is', 'Huawei', 'late', 'Dakota', '</s>', 'Magce', 'white', 'Dali', '<s>', 'Marta', 'NEC', 'young', 'funny', 'black', 'woman', 'Nokia', 'company', 'old', 'small', 'a', 'fat', 'sleeps'}


In [12]:
words

['<s>',
 'Roger',
 'is',
 'young',
 '</s>',
 '<s>',
 'Dali',
 'is',
 'a',
 'dog',
 '</s>',
 '<s>',
 'Dakota',
 'is',
 'old',
 '</s>',
 '<s>',
 'Roger',
 'is',
 'funny',
 '</s>',
 '<s>',
 'Marta',
 'is',
 'small',
 '</s>',
 '<s>',
 'Magce',
 'is',
 'fat',
 '</s>',
 '<s>',
 'Julia',
 'is',
 'white',
 '</s>',
 '<s>',
 'Julia',
 'is',
 'working',
 '</s>',
 '<s>',
 'Roger',
 'sleeps',
 'late',
 '</s>',
 '<s>',
 'Roger',
 'is',
 'happy',
 '</s>',
 '<s>',
 'NEC',
 'is',
 'a',
 'company',
 '</s>',
 '<s>',
 'Huawei',
 'is',
 'a',
 'company',
 '</s>',
 '<s>',
 'Nokia',
 'is',
 'a',
 'company',
 '</s>',
 '<s>',
 'Dakota',
 'is',
 'a',
 'dog',
 '</s>',
 '<s>',
 'Marta',
 'is',
 'a',
 'woman',
 '</s>',
 '<s>',
 'Magce',
 'is',
 'a',
 'woman',
 '</s>',
 '<s>',
 'Marta',
 'is',
 'black',
 '</s>']

In [14]:
def CountFrequency(my_list):

    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1

    for key, value in freq.items():
        print("% s : % d" % (key, value))

CountFrequency(words)


<s> :  17
Roger :  4
is :  16
young :  1
</s> :  17
Dali :  1
a :  7
dog :  2
Dakota :  2
old :  1
funny :  1
Marta :  3
small :  1
Magce :  2
fat :  1
Julia :  2
white :  1
working :  1
sleeps :  1
late :  1
happy :  1
NEC :  1
company :  3
Huawei :  1
Nokia :  1
woman :  2
black :  1


In [25]:
pair=[]
for i,j in zip(words,words[1:]):
    print(i,j)
    pair.append(i+j)

<s> Roger
Roger is
is young
young </s>
</s> <s>
<s> Dali
Dali is
is a
a dog
dog </s>
</s> <s>
<s> Dakota
Dakota is
is old
old </s>
</s> <s>
<s> Roger
Roger is
is funny
funny </s>
</s> <s>
<s> Marta
Marta is
is small
small </s>
</s> <s>
<s> Magce
Magce is
is fat
fat </s>
</s> <s>
<s> Julia
Julia is
is white
white </s>
</s> <s>
<s> Julia
Julia is
is working
working </s>
</s> <s>
<s> Roger
Roger sleeps
sleeps late
late </s>
</s> <s>
<s> Roger
Roger is
is happy
happy </s>
</s> <s>
<s> NEC
NEC is
is a
a company
company </s>
</s> <s>
<s> Huawei
Huawei is
is a
a company
company </s>
</s> <s>
<s> Nokia
Nokia is
is a
a company
company </s>
</s> <s>
<s> Dakota
Dakota is
is a
a dog
dog </s>
</s> <s>
<s> Marta
Marta is
is a
a woman
woman </s>
</s> <s>
<s> Magce
Magce is
is a
a woman
woman </s>
</s> <s>
<s> Marta
Marta is
is black
black </s>


In [28]:
e={}
for i in pair:
    if i in e:
        e[i]+=1
    else:
        e[i]=1

In [29]:
e

{'<s>Roger': 4,
 'Rogeris': 3,
 'isyoung': 1,
 'young</s>': 1,
 '</s><s>': 16,
 '<s>Dali': 1,
 'Daliis': 1,
 'isa': 7,
 'adog': 2,
 'dog</s>': 2,
 '<s>Dakota': 2,
 'Dakotais': 2,
 'isold': 1,
 'old</s>': 1,
 'isfunny': 1,
 'funny</s>': 1,
 '<s>Marta': 3,
 'Martais': 3,
 'issmall': 1,
 'small</s>': 1,
 '<s>Magce': 2,
 'Magceis': 2,
 'isfat': 1,
 'fat</s>': 1,
 '<s>Julia': 2,
 'Juliais': 2,
 'iswhite': 1,
 'white</s>': 1,
 'isworking': 1,
 'working</s>': 1,
 'Rogersleeps': 1,
 'sleepslate': 1,
 'late</s>': 1,
 'ishappy': 1,
 'happy</s>': 1,
 '<s>NEC': 1,
 'NECis': 1,
 'acompany': 3,
 'company</s>': 3,
 '<s>Huawei': 1,
 'Huaweiis': 1,
 '<s>Nokia': 1,
 'Nokiais': 1,
 'awoman': 2,
 'woman</s>': 2,
 'isblack': 1,
 'black</s>': 1}

In [41]:
e["Rogeris"]/sum(e.values())

0.03296703296703297